<a href="https://colab.research.google.com/github/ShraddhaSharma24/Natural-Language-Processing/blob/main/Fine_Tuning_BERT_for_Sentiment_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!pip install transformers datasets evaluate
!pip install scikit-learn


In [16]:
from datasets import load_dataset

dataset = load_dataset("glue", "sst2")
dataset["train"][0]


{'sentence': 'hide new secretions from the parental units ',
 'label': 0,
 'idx': 0}

In [17]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(example):
    return tokenizer(example["sentence"], padding="max_length", truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)


In [18]:
tokenized_dataset = tokenized_dataset.remove_columns(["sentence", "idx"])
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")
tokenized_dataset.set_format("torch")


In [19]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.01,
)


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [21]:
import evaluate

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)


In [22]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [23]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    report_to="none"  # <-- Disable W&B logging
)


In [25]:
from accelerate import Accelerator

Accelerator._reset_state()  # forcibly reset the state


AttributeError: type object 'Accelerator' has no attribute '_reset_state'

In [24]:
trainer.evaluate()


AttributeError: `AcceleratorState` object has no attribute `distributed_type`. This happens if `AcceleratorState._reset_state()` was called and an `Accelerator` or `PartialState` was not reinitialized.

In [26]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
import torch

# Load dataset
dataset = load_dataset("cnn_dailymail", "3.0.0", split={"train": "train[:1%]", "validation": "validation[:1%]"})

# Load tokenizer and model
model_name = "facebook/bart-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Preprocessing
def preprocess_function(examples):
    inputs = ["summarize: " + doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["highlights"], max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_train = dataset["train"].map(preprocess_function, batched=True)
tokenized_val = dataset["validation"].map(preprocess_function, batched=True)

# Training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_dir='./logs',
    report_to="none",  # disables wandb/logging
    save_total_limit=1,
)

# Define trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
)

# Train and evaluate
trainer.train()
trainer.evaluate()


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

...

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/134 [00:00<?, ? examples/s]

<ipython-input-26-9d1bda0a13e8>:41: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
test_article = dataset["validation"][0]["article"]
inputs = tokenizer("summarize: " + test_article, return_tensors="pt", truncation=True, max_length=512).to(model.device)
summary_ids = model.generate(inputs["input_ids"], max_length=150, min_length=40, length_penalty=2.0, num_beams=4)

print("\nGenerated Summary:\n", tokenizer.decode(summary_ids[0], skip_special_tokens=True))


In [28]:
from transformers import pipeline

# Load the summarization pipeline with a pretrained model
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# Your input text (can be anything from an article to a paragraph)
text = """
Harry Potter is a young wizard who discovers his magical heritage on his eleventh birthday.
He receives a letter of acceptance to Hogwarts School of Witchcraft and Wizardry and embarks
on an extraordinary journey filled with spells, magical creatures, and friendship.
"""

# Generate summary
summary = summarizer(text, max_length=60, min_length=20, do_sample=False)

# Display result
print("Summary:", summary[0]['summary_text'])


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu
Your max_length is set to 60, but your input_length is only 56. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=28)


Summary: Harry Potter is a young wizard who discovers his magical heritage on his eleventh birthday. He receives a letter of acceptance to Hogwarts School of Witchcraft and Wizardry.


In [29]:
from transformers import pipeline

qa_pipeline = pipeline("question-answering", model="distilbert-base-uncased-distilled-squad")

context = """
Harry Potter is a young wizard who attends Hogwarts School of Witchcraft and Wizardry.
He makes friends like Hermione Granger and Ron Weasley and faces many challenges,
including his battles with Lord Voldemort.
"""

question = "Who is Harry Potter's best friend?"

result = qa_pipeline(question=question, context=context)

print(result)


config.json:   0%|          | 0.00/451 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/265M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Device set to use cpu


{'score': 0.9398506879806519, 'start': 111, 'end': 143, 'answer': 'Hermione Granger and Ron Weasley'}


**Healthcare Applications**

In [1]:
from transformers import pipeline

# Use BioBERT / ClinicalBERT for medical NER or QA
qa = pipeline("question-answering", model="ktrapeznikov/biobert_v1.1_pubmed_squad_v2")

context = """The patient was admitted for chest pain. ECG showed ST elevation. Diagnosis was myocardial infarction."""
question = "What was the diagnosis?"

print(qa(question=question, context=context))



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of the model checkpoint at ktrapeznikov/biobert_v1.1_pubmed_squad_v2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cuda:0


{'score': 0.8888134360313416, 'start': 80, 'end': 101, 'answer': 'myocardial infarction'}


**Cyber-Physical Systems (CPS) + Security**

In [2]:
from transformers import pipeline

summarizer = pipeline("summarization")

logs = """
[INFO] User login from 192.168.1.1
[WARNING] Unusual activity detected
[ERROR] Failed login attempts from IP 10.0.0.1
"""

print(summarizer(logs, max_length=50, min_length=10, do_sample=False))


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

Device set to use cuda:0
Your max_length is set to 50, but your input_length is only 42. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=21)


[{'summary_text': ' User login from 192.168.1.1 . Unusual activity detected . Failed login attempts from IP 10.0.0 .1 .'}]


*** Explainable AI (XAI)***

In [3]:
from transformers import pipeline
from lime.lime_text import LimeTextExplainer

classifier = pipeline("sentiment-analysis")
explainer = LimeTextExplainer(class_names=["negative", "positive"])

def predict_proba(texts):
    preds = classifier(texts)
    return [[1 - p['score'], p['score']] for p in preds]

explanation = explainer.explain_instance("This medication has severe side effects", predict_proba, num_features=6)
explanation.show_in_notebook()


ModuleNotFoundError: No module named 'lime'